# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
clean_city_data=pd.read_csv('../Resources/Clean City Data.csv')
del clean_city_data['Unnamed: 0']

clean_city_data.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Punta Arenas,-53,-70.9167,46,93,90,2.30,CL,1619455831
1,Buena Vista,43,-83.8986,42,56,90,8.05,US,1619456079
2,Saskylakh,71,114.0833,-7,100,85,4.92,RU,1619456080
3,San Patricio,28,-97.5169,82,69,75,18.41,US,1619456080
4,Qaanaaq,77,-69.3632,37,74,72,6.76,GL,1619456081


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = clean_city_data[["Lat", "Lng"]]

# Fill NaN values and convert to float
weights = clean_city_data["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
# Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

ideal_weather=clean_city_data.loc[clean_city_data['Wind Speed']<10]
ideal_weather=ideal_weather.loc[ideal_weather['Max Temp']<90]
ideal_weather=ideal_weather.loc[ideal_weather['Max Temp']>80]
ideal_weather=ideal_weather.loc[ideal_weather['Cloudiness']<10]
ideal_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
178,Buraidah,26,43.9750,87,30,0,4.61,SA,1619456151
250,Balkanabat,39,54.3671,85,26,0,2.59,TM,1619456180
279,Ugoofaaru,5,73.0000,84,71,7,4.97,MV,1619456192
300,Kulhudhuffushi,6,73.0700,83,73,9,5.84,MV,1619456200
324,Sawākin,19,37.3321,85,66,1,6.42,SD,1619456210
368,Kavaratti,10,72.6420,85,70,0,7.58,IN,1619456139
369,Puerto Escondido,15,-97.0667,86,70,1,7.78,MX,1619456229
401,Makkah al Mukarramah,21,39.8261,87,37,0,6.93,SA,1619456243
442,Piedra Gorda,18,-66.8877,86,65,1,8.05,PR,1619456260
495,Datia,25,78.4667,89,5,0,1.97,IN,1619456282


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df=ideal_weather

In [8]:
# geocoordinates
target_radius = 5000
target_search = "hotel"
key_search = "hotel"
# set up a parameters dictionary

params = {
#     "location": target_coordinates,
    "radius": target_radius,
    "types": target_search,
    "keyword": key_search,
    "key": g_key}

for index, row in hotel_df.iterrows():
    
    # latitudes and longitudes from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
#     print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
178,Buraidah,26,43.9750,87,30,0,4.61,SA,1619456151,Swiss International Resort Unaizah - Al Qassim
250,Balkanabat,39,54.3671,85,26,0,2.59,TM,1619456180,NaN
279,Ugoofaaru,5,73.0000,84,71,7,4.97,MV,1619456192,Finolhu
300,Kulhudhuffushi,6,73.0700,83,73,9,5.84,MV,1619456200,NaN
324,Sawākin,19,37.3321,85,66,1,6.42,SD,1619456210,NaN
368,Kavaratti,10,72.6420,85,70,0,7.58,IN,1619456139,NaN
369,Puerto Escondido,15,-97.0667,86,70,1,7.78,MX,1619456229,NaN
401,Makkah al Mukarramah,21,39.8261,87,37,0,6.93,SA,1619456243,NaN
442,Piedra Gorda,18,-66.8877,86,65,1,8.05,PR,1619456260,Copamarina Beach Resort & Spa
495,Datia,25,78.4667,89,5,0,1.97,IN,1619456282,Tomar Hotel Bamhorisar


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)
fig
# Display figure


Figure(layout=FigureLayout(height='420px'))